# A simple basin filling

This example shows the filling of a small-scale basin induced simply by an uniform precipitation and sea level change. 

The initial grid is a depression surface scaled by _30_ km by _30_ km with a resolution of _100_ m. The elevation ranges from _-100_ m to _300_ m. This case runs to _1000k_ years with an output interval of _10k_ year. A uniform precipitation rate of _3_ m/a is applied to the whole area. The sea-level is simply modelled using a sine curve. 

|<img src="images/geometry.png" alt="geometry" width="400" height="200"/>| <img src="images/sealevel.png" alt="sea level" width="250" height="200"/> |
|:-:|:-:|
|Initial surface (black lines are initial shoreline positions)|Sea level curve|

In this case, we will present the workflow of stratal analysis of the depositional architecture. The stratal stacking petterns will be produced firstly, based on which the Wheeler diagram and "well logs" will then be obtained. In addition, three analysis methods can be applied to interpret the resulting stratigraphic sequences.

# 1. Generate the topographic grid

Here, we use a sine wave function to generate the node file of the initial surface.

The file provides for each line the following information:
+ X coordinates in meters (this axis has a West to East orientation),
+ Y coordinates in meters (this axis has a South to West orientation), 
+ Z coordinates in meters.

**Note:** Nodes must be defined in increasing order from the South/West corner, first along the X axis.

We create a sea level curve from scratch.

In [ ]:
%matplotlib inline

# Import badlands grid generation toolbox
import badlands_companion.toolGeo as simple

# display plots in SVG format
%config InlineBackend.figure_format = 'svg' 

We first initialize the extent of the simulation area.

The function takes 3 arguments:
+ the extent of the domain along the X-axis [xmin,xmax] in metres,
+ the extent of the domain along the Y-axis [ymin,ymax] in metres,
+ the grid spacing `dx` in metres.

In [ ]:
# help(simple.toolGeo.__init__)
wave = simple.toolGeo(extentX=[0.,30000.], extentY=[0.,30000.], dx=100.)

We create a wave by defining 3 parameters:

+ the amplitude of the wave: A in metres,
+ the period of the wave: P in metres,
+ the base of the cumulative displacement in metres,
+ the center of the wave.
+ the axis used to build the wave function (either the X or Y axis)

For more information regarding the function uncomment the following cell.

In [ ]:
# help(wave.buildWave)
wave.Z = wave.buildWave(A=-400., P=30000., base=300., xcenter=15000.)

In [ ]:
# It is possible to have a look at the grid surface 
# using **plotly** library before proceeding to the creation of the badlands surface.
# 
# help(tecwave.dispView)
wave.viewGrid(width=600, height=600, zmin=-500, zmax=500, zData=wave.Z, title='Export Grid')

Once the surface has been generated, we export a **CSV** regular grid that will be loaded in the simulation.

In [ ]:
# help(wave.buildGrid)
wave.buildGrid(elevation=wave.Z, nameCSV='data/node')

# 2. Building sea-level curve

The file (**CSV format**) provides for each line the following information:
* Time in years,
* Sea-level elevation for the considered time in meters.

In addition, the defined fluctuation times should be set in **increasing order starting from the oldest time**.

We create a sea level curve from scratch. For more details please refer to [/workspace/companion/seaLevel.ipynb](/workspace/companion/seaLevel.ipynb).

In [ ]:
%matplotlib inline

# Import badlands grid generation toolbox
import badlands_companion.toolSea as tools

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'

In [ ]:
sea = tools.toolSea()
sea.buildCurve(timeExt = [0.,1000000.], timeStep = 10000., seaExt = [0.,0.], 
                   ampExt = [20.,20.], periodExt = [500000.,500000.])
# Visualize
sea.plotCurves(fsize=(4,5), figName = 'Sea level')

In [ ]:
# Export the sea-level file  to the data folder
sea.exportCurve(nameCSV='data/sealevel_1myr')

# 3. Running badlands

We can run the model for a given period. The end time in the **XmL** input file is set to _500k years_ but you might want to run the model for a coupled of iterations and check the output before running the model for the entire simulation time. 

This is done by putting the time in the `run_to_time` function. 

Here we go for the full time directly... 

In [ ]:
from badlands.model import Model as badlandsModel

# initialise model
model = badlandsModel()
# Define the XmL input file
model.load_xml('basin.xml')

In [ ]:
model.run_to_time(1000000)